In [8]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 21: Scrambled Letters and Hash ---</h2><p>The computer system you're breaking into uses a <span title="I do not like them, Security-Account-Manager-I-Am! I do not like scrambled letters and hash!">weird scrambling function</span> to store its passwords. It shouldn't be much trouble to create your own scrambled password so you can add it to the system; you just have to implement the scrambler.</p>
<p>The scrambling function is a series of operations (the exact list is provided in your puzzle input). Starting with the password to be scrambled, apply each operation in succession to the string. The individual operations behave as follows:</p>
<ul>
<li><code>swap position X with position Y</code> means that the letters at indexes <code>X</code> and <code>Y</code> (counting from <code>0</code>) should be <em>swapped</em>.</li>
<li><code>swap letter X with letter Y</code> means that the letters <code>X</code> and <code>Y</code> should be <em>swapped</em> (regardless of where they appear in the string).</li>
<li><code>rotate left/right X steps</code> means that the whole string should be <em>rotated</em>; for example, one right rotation would turn <code>abcd</code> into <code>dabc</code>.</li>
<li><code>rotate based on position of letter X</code> means that the whole string should be <em>rotated to the right</em> based on the <em>index</em> of letter <code>X</code> (counting from <code>0</code>) as determined <em>before</em> this instruction does any rotations.  Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least <code>4</code>.</li>
<li><code>reverse positions X through Y</code> means that the span of letters at indexes <code>X</code> through <code>Y</code> (including the letters at <code>X</code> and <code>Y</code>) should be <em>reversed in order</em>.</li>
<li><code>move position X to position Y</code> means that the letter which is at index <code>X</code> should be <em>removed</em> from the string, then <em>inserted</em> such that it ends up at index <code>Y</code>.</li>
</ul>
<p>For example, suppose you start with <code>abcde</code> and perform the following operations:</p>
<ul>
<li><code>swap position 4 with position 0</code> swaps the first and last letters, producing the input for the next step, <code>ebcda</code>.</li>
<li><code>swap letter d with letter b</code> swaps the positions of <code>d</code> and <code>b</code>: <code>edcba</code>.</li>
<li><code>reverse positions 0 through 4</code> causes the entire string to be reversed, producing <code>abcde</code>.</li>
<li><code>rotate left 1 step</code> shifts all letters left one position, causing the first letter to wrap to the end of the string: <code>bcdea</code>.</li>
<li><code>move position 1 to position 4</code> removes the letter at position <code>1</code> (<code>c</code>), then inserts it at position <code>4</code> (the end of the string): <code>bdeac</code>.</li>
<li><code>move position 3 to position 0</code> removes the letter at position <code>3</code> (<code>a</code>), then inserts it at position <code>0</code> (the front of the string): <code>abdec</code>.</li>
<li><code>rotate based on position of letter b</code> finds the index of letter <code>b</code> (<code>1</code>), then rotates the string right once plus a number of times equal to that index (<code>2</code>): <code>ecabd</code>.</li>
<li><code>rotate based on position of letter d</code> finds the index of letter <code>d</code> (<code>4</code>), then rotates the string right once, plus a number of times equal to that index, plus an additional time because the index was at least <code>4</code>, for a total of <code>6</code> right rotations: <code>decab</code>.</li>
</ul>
<p>After these steps, the resulting scrambled password is <code>decab</code>.</p>
<p>Now, you just need to generate a new scrambled password and you can access the system. Given the list of scrambling operations in your puzzle input, <em>what is the result of scrambling <code>abcdefgh</code></em>?</p>
</article>


In [9]:
example_operations = """
swap position 4 with position 0
swap letter d with letter b
reverse positions 0 through 4
rotate left 1 step
move position 1 to position 4
move position 3 to position 0
rotate based on position of letter b
rotate based on position of letter d
"""

password = "abcde"
expected = "decab"


def scramble(password: str, operations: str) -> str:
    pw = list(password)

    for line in operations.strip().splitlines():
        if line.startswith("swap position"):
            # swap position X with position Y
            x, y = map(int, re.findall(r"\d+", line))
            pw[x], pw[y] = pw[y], pw[x]
        elif line.startswith("swap letter"):
            # swap letter X with letter Y
            x, y = re.findall(r"\b[a-z]{1}\b", line)
            xi = pw.index(x)
            yi = pw.index(y)
            pw[xi], pw[yi] = pw[yi], pw[xi]
        elif line.startswith("rotate left"):
            # rotate left X steps
            x, *_ = map(int, re.findall(r"\d+", line))
            pw = pw[x:] + pw[:x]
        elif line.startswith("rotate right"):
            # rotate right X steps
            x, *_ = map(int, re.findall(r"\d+", line))
            pw = pw[-x:] + pw[:-x]
        elif line.startswith("rotate based"):
            # rotate based on position of letter X means
            x, *_ = re.findall(r"\b[a-z]{1}\b", line)
            xi = pw.index(x)
            pw = pw[-xi - 1 :] + pw[: -xi - 1]
            if xi > 3:
                pw = pw[-1:] + pw[:-1]

        elif line.startswith("reverse positions"):
            # reverse positions X through Y
            x, y = map(int, re.findall(r"\d+", line))
            pw[x : y + 1] = pw[y : x - 1 : -1] if x else pw[y::-1]

        elif line.startswith("move position"):
            # move position X to position Y
            x, y = map(int, re.findall(r"\d+", line))
            l = pw.pop(x)
            pw.insert(y, l)
        else:
            raise ValueError(f"Unknown operations {line}.")

    return "".join(pw)


scramble(password, example_operations), expected

('decab', 'decab')

In [10]:
with open("../input/day21.txt") as f:
    operations = f.read()


print("Part I:", scramble("abcdefgh", operations))

Part I: bdfhgeca


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>bdfhgeca</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You scrambled the password correctly, but you discover that you <a href="https://en.wikipedia.org/wiki/File_system_permissions">can't actually modify</a> the <a href="https://en.wikipedia.org/wiki/Passwd">password file</a> on the system. You'll need to un-scramble one of the existing passwords by reversing the scrambling process.</p>
<p>What is the un-scrambled version of the scrambled password <code>fbgdceah</code>?</p>
</article>

</main>


In [13]:
def unscramble(scrambeled: str, operations: str):
    pw = list(scrambeled)

    for line in reversed(operations.strip().splitlines()):
        if line.startswith("swap position"):
            # swap position X with position Y
            x, y = map(int, re.findall(r"\d+", line))
            pw[x], pw[y] = pw[y], pw[x]
        elif line.startswith("swap letter"):
            # swap letter X with letter Y
            x, y = re.findall(r"\b[a-z]{1}\b", line)
            xi = pw.index(x)
            yi = pw.index(y)
            pw[xi], pw[yi] = pw[yi], pw[xi]
        elif line.startswith("rotate left"):
            # rotate left X steps
            x, *_ = map(int, re.findall(r"\d+", line))
            pw = pw[-x:] + pw[:-x]
        elif line.startswith("rotate right"):
            # rotate right X steps
            x, *_ = map(int, re.findall(r"\d+", line))
            pw = pw[x:] + pw[:x]
        elif line.startswith("rotate based"):
            # rotate based on position of letter X means
            x, *_ = re.findall(r"\b[a-z]{1}\b", line)
            xi = pw.index(x)

            rotations = -1
            if xi % 2:
                rotations = -(xi + 1) // 2
            elif xi:
                rotations = (len(pw) - xi - 2) // 2

            pw = pw[-rotations:] + pw[:-rotations]
        elif line.startswith("reverse positions"):
            # reverse positions X through Y
            x, y = map(int, re.findall(r"\d+", line))
            pw[x : y + 1] = pw[y : x - 1 : -1] if x else pw[y::-1]

        elif line.startswith("move position"):
            # move position X to position Y
            x, y = map(int, re.findall(r"\d+", line))
            l = pw.pop(y)
            pw.insert(x, l)
        else:
            raise ValueError(f"Unknown operations {line}.")

    return "".join(pw)


print(unscramble(scramble("abcde", example_operations), example_operations))
print(unscramble(scramble("abcdefgh", operations), operations))

abcde
abcdefgh


In [14]:
print("Part II:", unscramble("fbgdceah", operations))

Part II: gdfcabeh


<link href="style.css" rel="stylesheet"></link>

<p>Your puzzle answer was <code>gdfcabeh</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
